In [1]:
import utils
import importlib

importlib.reload(utils)

gpu_ids = [6]
utils.set_gpu_ids(gpu_ids)

os.environ["CUDA_VISIBLE_DEVICES"] = 6


In [2]:
import transformers
import os
import torch

In [3]:
model = utils.get_hf_model(
    "/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf",
)

[2023-08-24 00:59:06,749] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>
LlamaConfig {
  "_name_or_path": "/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32001
}



In [4]:
print(type(model))
print(model.config)

<class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>
LlamaConfig {
  "_name_or_path": "/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32001
}



In [5]:
tokenizer = utils.get_complete_tokenizer()

LlamaTokenizerFast(name_or_path='/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--hf-internal-testing--llama-tokenizer/snapshots/99eceeba6e8289bee767f0771166b5917e70e470', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)


In [6]:
problem_solution_example = {
    "problem": "A $90^\\circ$ rotation around the origin in the counter-clockwise direction is applied to $7 + 2i.$  What is the resulting complex number?",
    "steps": [
        "I know that a $90^\\circ$ rotation around the origin in the complex plane can be achieved by multiplying the complex number by $i$, since $i$ has a magnitude of 1 and an argument of $90^\\circ$.",
        "So, I can write the rotation as $(7 + 2i)i = 7i + 2i^2.$",
        "To simplify this expression, I recall that $i^2 = -1$, so I can substitute that and get $7i - 2.$",
        "This is the resulting complex number after the rotation.\n\n# Answer\n\n7i - 2",
    ],
}

In [7]:
solution_till_step = utils.problem_steps2solution_till_step_list(
    problem_solution_example["problem"], problem_solution_example["steps"]
)
print(solution_till_step[-1])

A $90^\circ$ rotation around the origin in the counter-clockwise direction is applied to $7 + 2i.$  What is the resulting complex number?
I know that a $90^\circ$ rotation around the origin in the complex plane can be achieved by multiplying the complex number by $i$, since $i$ has a magnitude of 1 and an argument of $90^\circ$.
So, I can write the rotation as $(7 + 2i)i = 7i + 2i^2.$
To simplify this expression, I recall that $i^2 = -1$, so I can substitute that and get $7i - 2.$
This is the resulting complex number after the rotation.

# Answer

7i - 2


In [15]:
inputs = utils.preprocess(tokenizer, solution_till_step[-1]).to("cuda:0")
# print(f"inputs = {inputs}")
# model.eval()
print(f"inputs.input_ids.shape = {inputs.input_ids.shape}")

AttributeError: 

In [13]:
outputs = model.forward(
    **inputs,
    # generation_config=utils.generation_config
)
print(f"outputs = {outputs}")

outputs = CausalLMOutputWithPast(loss=None, logits=tensor([[[ -4.9414,   0.8208,   2.2891,  ...,  -0.4036,  -1.4893,  -2.8652],
         [-11.6328,  -9.1172,  -3.0156,  ...,  -8.4766,  -8.3359,  -1.3770],
         [ -7.0586,  -7.2539,   0.7959,  ...,  -4.7617,  -4.0000,  -1.2217],
         ...,
         [ -4.3438,  -4.0195,   3.1602,  ...,  -4.0352,  -3.8184,  -3.9922],
         [ -3.5898,  -3.1035,   3.5742,  ...,  -1.5986,  -0.7876,  -1.6309],
         [ -3.6016,  -3.2793,   3.6367,  ...,  -3.7285,  -3.3887,  -4.3867]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>), past_key_values=((tensor([[[[-0.4575,  0.0063,  0.0547,  ...,  0.1049, -0.0908, -0.0728],
          [-0.2101, -0.0829,  0.0423,  ...,  0.2273, -0.1582,  0.2233],
          [ 0.0293, -0.1544, -0.3105,  ...,  0.5225, -0.4094,  0.5352],
          ...,
          [ 0.5835,  0.2334, -0.2747,  ...,  0.5479, -0.3909,  0.7417],
          [-0.6431, -0.7070,  0.5259,  ..., -0.3186,  0.3115, -0.2983],
          [-0.3223, -0.158

In [14]:
print(type(outputs))
logits = outputs.logits
print(f"logits.shape = {logits.shape}")
probs = torch.softmax(logits, dim=-1)
print(f"probs.shape = {probs.shape}")
# print(f"probs = {probs}")
max_prob_indices = torch.argmax(probs, dim=-1)
print(f"max_prob_indices = {max_prob_indices}")
tokenizer.decode(max_prob_indices[0].tolist())

# base model outputs
# last_hidden_state = outputs.last_hidden_state

<class 'transformers.modeling_outputs.CausalLMOutputWithPast'>
logits.shape = torch.Size([1, 170, 32001])
probs.shape = torch.Size([1, 170, 32001])
max_prob_indices = tensor([[23196,   716, 29896,  6374,  8178,  8178,  8178,  8178,  6374,  6374,
          6374,  6374,  6374,  6374,  6374,  6374,  6374,  6374,  6374,  6374,
          6374,  6374,  1066,  8178,  8178,  8178,  8178,  8178,  6374,  6374,
          6374,  8178,  6374,  6374,  6374,  6374,  6374,  6374,  8178,  6374,
          6374,  6374,  6374,  6374,  6374,  6374,  6374,  6374,  6374,  6374,
          6374,  6374,  6374,  6374,  6374,  6374,  6374,  6374,  6374,  6374,
          5890,  6374,  6374,  6374,  6374,  6374,  6374,  8178,  8178,  8178,
          8178,  8178,  8178,  8178,  8178,  6374,  6374,  6374,  6374,  6374,
          6374,  6374,  6374,  6374,  6374,  6374,  6374,  6374,  8178,  8178,
          8178,  8178,  8178,  6374,  6374,  6374,  6374,  6374,  6374,  6374,
          6374,  6374,  6374,  8178,  8178,

'nobody new1 positive negative negative negative negative positive positive positive positive positive positive positive positive positive positive positive positive positive positivepos negative negative negative negative negative positive positive positive negative positive positive positive positive positive positive negative positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positive positivelying positive positive positive positive positive positive negative negative negative negative negative negative negative negative positive positive positive positive positive positive positive positive positive positive positive positive positive negative negative negative negative negative positive positive positive positive positive positive positive positive positive positive negative negative positive positive positive negative negative positive2 positive positive positi

In [ ]:
print(outputs.last_hidden_state.shape)

torch.Size([1, 170, 4096])


In [10]:
for k, v in outputs.items():
    print(f"{k}")

logits
past_key_values
